# Restaurant Opening

Business Problem

One of my friend is planing to open a restaurant in Toronto and asked my opinion where should she open it. She have some concerns about the locatition where she should open the restaurant because she doesn't have any other capital to close and open on another location. The opening has to be perfect.

I suggested her that we should look the city data and consider the results to give us idea.

By segmenting neighborhoods of Toronto by the number of existing food and drink locations we could try to determine current competition level. It would be wise to narrow down our choises by trying to focus on neighborhoods with low level of competition.

In order to that we will try to get neighborhood data from Wikipedia web site and enrich the data with foursquare venue data.